In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from postgres import connection
import psycopg2
from datetime import datetime
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2023-09-08 12:24:23.973216: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-08 12:24:24.176189: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-08 12:24:25.179849: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [101]:
def download_data(connection: psycopg2.extensions.connection, start_date = "2009-12-01", end_date = "2022-12-30") -> pd.DataFrame:
    cursor = connection.cursor()
    cursor.execute("SELECT ticker FROM equities")
    tickers = cursor.fetchall()
    stock_data = {}
    null_volumes = {}

    for stock_tuple in tickers:
        stock = stock_tuple[0]
        if stock == "^STI":
            continue
        table_name = f"stock_{stock[:3]}"  # Remove the .SI suffix
        query = (
            f"SELECT * FROM {table_name} WHERE Date >= %s AND Date <= %s ORDER BY Date ASC"
        )
        cursor.execute(query, (start_date, end_date))
        data = cursor.fetchall()
        if data:
            dates, open_price, high, low, close, adj_close, volume = zip(*data)
            
            if datetime.strptime(start_date, "%Y-%m-%d") != datetime.strptime(dates[0], "%Y-%m-%d") \
                or datetime.strptime(end_date, "%Y-%m-%d") != datetime.strptime(dates[-1], "%Y-%m-%d"):
                continue

            df = pd.DataFrame({
                'Date': pd.to_datetime(dates),
                'Open': open_price,
                'High': high,
                'Low': low,
                'Close': close,
                'Adj_Close': adj_close,
                'Volume': volume
            })
            if df[-1:]['Adj_Close'].values[0] < 0.2:
                continue
            # Handle special case where all columns are equal
            mask = (df['Open'] == df['High']) & (df['High'] == df['Low']) & (df['Low'] == df['Close']) & (df['Close'] == df['Adj_Close'])
            df.loc[mask, 'Volume'] = -1.0
            
            df.replace(0, np.nan, inplace=True)
            
            null_counts = df.isnull().sum()
            if null_counts.any():
                null_volumes[stock] = {'null_count': null_counts['Volume'], '0_vol': df[df['Volume'] == -1.0].shape[0]}  
            stock_data[stock] = df

    cursor.close()
    return stock_data, null_volumes


In [102]:
all_data,null_volumes = download_data(connection)

In [105]:
null_volumes_df = pd.DataFrame.from_dict(null_volumes, orient='index', columns=[ 'null_count', '0_vol'])
null_volumes_df.sort_values(by=['null_count'], ascending=False, inplace=True)
null_volumes_df.to_excel('null_volumes.xlsx')

In [94]:
test_data = {}
training_data = {}
for stock, df in all_data.items():
    if stock in null_volumes:
        test_data[stock] = df
    else:
        training_data[stock] = df
        
print(f"Number of stocks in training data: {len(training_data)}")
print(f"Number of stocks in test data: {len(test_data)}")

Number of stocks in training data: 0
Number of stocks in test data: 97


In [95]:
for key, value in training_data.items():
    print(key)
    print(value[value['Volume'] == -1])

In [4]:
connection.close()